<a href="https://colab.research.google.com/github/Raj-GMITS/flutter-ios-useful-commands/blob/main/stable_diffusion_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
!pip install diffusers transformers huggingface_hub requests flask

/bin/bash: line 1: nvidia-smi: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.0 MB/s eta 0:00:00


In [ ]:
!pip install firebase-admin

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [ ]:
pipe = pipe.to("cuda")

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
prompt = "a photograph of an astronaut riding a horse"
image = pipe(prompt).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)

# Now to display an image you can either save it such as:
image.save(f"astronaut_rides_horse.png")

# or if you're in a google colab you can directly display it with
image

In [ ]:
import json
import firebase_admin
from firebase_admin import credentials, db, storage
from datetime import datetime
import time


cred = credentials.Certificate("cred.json")

# Initialize Firebase Admin SDK with a unique app name
try:
    app = firebase_admin.initialize_app(cred, {'databaseURL': 'https://sync-paw-default-rtdb.asia-southeast1.firebasedatabase.app','storageBucket': 'sync-paw.appspot.com'}, name='sync-paw')
except ValueError:
    # If the app is already initialized, catch the ValueError and continue
    app = firebase_admin.get_app('sync-paw')

# Define Firebase Realtime Database reference
ref = db.reference(app=app, url='https://sync-paw-default-rtdb.asia-southeast1.firebasedatabase.app')

# Define Firebase Storage bucket
bucket = storage.bucket(app=app,name='sync-paw.appspot.com')

# Define the generate_image function
def generate_image(prompt_value, num_inference_steps=50, guidance_scale=7.5):
    try:
        # Generate a timestamp to be used as part of the image name
        timestamp = datetime.now().strftime('%Y%m%d%H%M%S')

        image = pipe(prompt_value).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)
        print(f"{timestamp}.png")
        # Now to display an image you can either save it such as:
        image.save(f"{timestamp}.png")

        # # Assuming 'pipe' is some function you've defined elsewhere
        # image = pipe(prompt=prompt_value, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]

        # # Convert the PyTorch tensor to a list
        # image_list = image.cpu().numpy().tolist()

        # Upload the generated image to Firebase Storage
        # blob = bucket.blob(f"{time.clock_gettime}.png")
        # blob.upload_from_string(json.dumps(image_list), content_type='application/json')

        blob = bucket.blob(f"generated_images/{timestamp}.png")
        blob.upload_from_filename(f"{timestamp}.png")

        # Get the URL of the uploaded image
        image_url = blob.public_url

        print(f'Image uploaded successfully. URL: {image_url}')

        # Trigger the 'generated_image' event in Realtime Database
        ref.update({f"{timestamp}.png":image_url})

        # return {"image": image_list, "image_url": image_url}

    except Exception as e:
        return {"error": str(e)}

# Set up Realtime Database event listener
def on_prompt_change(event):
    prompt_value = event.data
    if prompt_value is not None and 'prompt' in prompt_value:
        prompt_text = prompt_value['prompt']
        print(f"Prompt value changed to: {prompt_text}")

        # Call the generate_image function or perform any other actions based on the prompt value
        result = generate_image(prompt_text)
        print(result)

# # Listen for changes in the 'prompt' value
# db.reference(app=app, url='https://sync-paw-default-rtdb.asia-southeast1.firebasedatabase.app/prompt').listen(on_prompt_change)

# Listen for changes in the 'prompt' value
prompt_ref = ref.child('prompt')
# listener = prompt_ref.listen(on_prompt_change)

def on_data_change(snapshot):
    data = snapshot.data
    print(f"Prompt value changed to: {data}")

    # Call the generate_image function or perform any other actions based on the prompt value
    result = generate_image(data)
    print(result)


prompt_ref.listen(on_data_change)


# Trigger the 'prompt' value change in Realtime Database (for testing)
ref.update({'prompt':'A Beautiful little cat'})


In [ ]:
# import json
# import firebase_admin
# from firebase_admin import credentials, db, storage
# from datetime import datetime
# cred = credentials.Certificate("cred.json")

# # Initialize Firebase Admin SDK with a unique app name
# try:
#     app = firebase_admin.initialize_app(cred, {'databaseURL': 'https://sync-paw-default-rtdb.asia-southeast1.firebasedatabase.app','storageBucket': 'sync-paw.appspot.com'}, name='sync-paw')
# except ValueError:
#     # If the app is already initialized, catch the ValueError and continue
#     app = firebase_admin.get_app('sync-paw')

# # Define Firebase Realtime Database reference
# ref = db.reference(app=app, url='https://sync-paw-default-rtdb.asia-southeast1.firebasedatabase.app')

# # Define Firebase Storage bucket
# bucket = storage.bucket(app=app,name='sync-paw.appspot.com')

# # Define the generate_image function
# def generate_image(prompt_value, num_inference_steps=50, guidance_scale=7.5):
#     try:
#         # Generate a timestamp to be used as part of the image name
#         timestamp = datetime.now().strftime('%Y%m%d%H%M%S')

#         image = pipe(prompt_value).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)
#         print(f"{timestamp}.png")
#         # Now to display an image you can either save it such as:
#         image.save(f"{timestamp}.png")

#         # # Assuming 'pipe' is some function you've defined elsewhere
#         # image = pipe(prompt=prompt_value, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]

#         # # Convert the PyTorch tensor to a list
#         # image_list = image.cpu().numpy().tolist()

#         # Upload the generated image to Firebase Storage
#         # blob = bucket.blob(f"{time.clock_gettime}.png")
#         # blob.upload_from_string(json.dumps(image_list), content_type='application/json')

#         blob = bucket.blob(f"generated_images/{timestamp}.png")
#         blob.upload_from_filename(f"{timestamp}.png")

#         # Get the URL of the uploaded image
#         image_url = blob.public_url

#         print(f'Image uploaded successfully. URL: {image_url}')

#         # Trigger the 'generated_image' event in Realtime Database
#         db.reference(app=app, url='https://sync-paw-default-rtdb.asia-southeast1.firebasedatabase.app').put('/events', 'generated_image', {'image_url': image_url})

#         # return {"image": image_list, "image_url": image_url}

#     except Exception as e:
#         return {"error": str(e)}

# # Set up Realtime Database event listener
# def on_prompt_change(event):
#     prompt_value = event.data
#     print(f"Prompt value changed to: {prompt_value}")

#     # Call the generate_image function or perform any other actions based on the prompt value
#     result = generate_image(prompt_value['prompt'])
#     print(result)

# # Listen for changes in the 'prompt' value
# db.reference(app=app, url='https://sync-paw-default-rtdb.asia-southeast1.firebasedatabase.app/prompt').listen(on_prompt_change)

# # Trigger the 'prompt' value change in Realtime Database (for testing)
# ref.set({'prompt':'A Beautiful little cat'})
